In [3]:
using LinearAlgebra, Plots, LaTeXStrings, Symbolics 

In [4]:
const E = 2.1E06
const ν = 0.3

global C = ((1 - ν) / ((1 + ν) * (1 - ν))) * E * [1 (ν/(1-ν)) 0; (ν/(1-ν)) 1 0; 0 0 ((1-2*ν)/(2*(1-ν)))];

In [5]:
const numNodosx = 3
const numNodosy = 3
const numNodosGlobales = numNodosx * numNodosy

global GDLGlobales = numNodosGlobales * 2  # En el caso de elasticidad bidimensional se tiene 2 grados de libertad por nodo

const longx = 3.0
const longy = 3.0

const numElementos = (numNodosx - 1) * (numNodosy - 1);

In [6]:
# Matriz de conectividad global para elementos bidimensionales de cuatro nodos 
#(El numero de elementos podría variar)

function matrizConectividadGlobal(numNodosx, numNodosy, numElementos)
    matrizConectividad = zeros(Int, numElementos, 4)
    elemento = 1
    for j in 1:numNodosy-1
        for i in 1:numNodosx-1
            nodoInferiorIzquierdo::Int = i + ((j - 1) * numNodosx)
            matrizConectividad[elemento, :] = [(nodoInferiorIzquierdo + numNodosx + 1) (nodoInferiorIzquierdo + numNodosx) nodoInferiorIzquierdo (nodoInferiorIzquierdo + 1)]
            elemento += 1
        end
    end
    return matrizConectividad
end 


matrizConectividadGlobal (generic function with 1 method)

In [7]:
function mallador(numNodosx, numNodosy, longx, longy, numElementos)

    numSegmentosx = numNodosx - 1
    numSegmentosy = numNodosy - 1
    dNodosx = longx / numSegmentosx
    dNodosy = longy / numSegmentosy
    numNodosGlobales = numNodosx * numNodosy

    matrizCoordGlobales = zeros(numNodosGlobales, 2)
    Coordy = 0
    nodo = 1
    for j in 1:numNodosy
        Coordx = 0
        for i in 1:numNodosx
            matrizCoordGlobales[nodo, :] = [Coordx Coordy]
            Coordx += dNodosx
            nodo += 1
        end
        Coordy += dNodosy
    end

    return matrizCoordGlobales
end

mallador (generic function with 1 method)

In [8]:
function matrizGdGL(numNodosx, numNodosy, mConectividadGlobal)
    numElementos = (numNodosx - 1) * (numNodosy - 1)
    numNodosGlobales = numNodosx * numNodosy
    mGdGL = zeros(Integer, 8, numElementos)
    for i in 1:numElementos
        j = 1
        while j <= 4
            zz = mConectividadGlobal[i, :]
            mGdGL[2*j-1, i] = zz[j] * 2 - 1
            mGdGL[2*j, i] = zz[j] * 2
            j += 1
        end
    end
    return mGdGL
end

matrizGdGL (generic function with 1 method)

In [9]:
mConectividadGlobal = matrizConectividadGlobal(numNodosx, numNodosy, numElementos)
#display(mConectividadGlobal)
mGdLG = matrizGdGL(numNodosx, numNodosy, mConectividadGlobal)
display(mGdLG)
mCoordNodales = mallador(numNodosx, numNodosy, longx, longy, numElementos)
#display(mCoordNodales)

8×4 Matrix{Integer}:
  9  11  15  17
 10  12  16  18
  7   9  13  15
  8  10  14  16
  1   3   7   9
  2   4   8  10
  3   5   9  11
  4   6  10  12

9×2 Matrix{Float64}:
 0.0  0.0
 1.5  0.0
 3.0  0.0
 0.0  1.5
 1.5  1.5
 3.0  1.5
 0.0  3.0
 1.5  3.0
 3.0  3.0

In [10]:
begin
    @variables r s
    Dr = Differential(r)
    Ds = Differential(s)

    nFInt = 4
    h₁ = 1 / 4 * (1 + r) * (1 + s)
    h₂ = 1 / 4 * (1 - r) * (1 + s)
    h₃ = 1 / 4 * (1 - r) * (1 - s)
    h₄ = 1 / 4 * (1 + r) * (1 - s)

    H = [h₁ h₂ h₃ h₄]

    ∇H = [transpose([expand_derivatives(Dr(H[i])) for i in 1:nFInt]); transpose([expand_derivatives(Ds(H[i])) for i in 1:nFInt])]
end

2×4 Matrix{Num}:
 0.25(1 + s)  -0.25(1 + s)  -0.25(1 - s)   0.25(1 - s)
 0.25(1 + r)   0.25(1 - r)  -0.25(1 - r)  -0.25(1 + r)

In [11]:
nNodosInt = 9
GL9Q = [-sqrt(0.6) -sqrt(0.6); 0 -sqrt(0.6); sqrt(0.6) -sqrt(0.6); sqrt(0.6) 0; sqrt(0.6) sqrt(0.6);
    0 sqrt(0.6); -sqrt(0.6) sqrt(0.6); -sqrt(0.6) 0; 0 0]
#display(GL9Q)
wGL9Q = [5/9 5/9; 8/9 5/9; 5/9 5/9; 5/9 8/9; 5/9 5/9; 8/9 5/9; 5/9 5/9; 5/9 8/9; 8/9 8/9];

In [12]:
function matrizKGL(mConectividadGlobal, mCoordNodales, C, numElementos, GDLGlobales, mGdLG, GL9Q, wGL9Q)

    KG = zeros(Float64, GDLGlobales, GDLGlobales) 
    mNodalCoords = zeros(4, 2)

    for el in 1:numElementos

        KE = zeros(Float64, 8, 8)
        GLE = mGdLG[:, el]
        numGLE = length(GLE)
        mConectividadElemental = mConectividadGlobal[el, :]

        for nodo in 1:4
         mNodalCoords[nodo, :] = mCoordNodales[mConectividadGlobal[el, :][nodo], :]
            #mNodalCoords[nodo, :] = mCoordNodales[mConectividadElemental[nodo], :]
        end

        x = expand((H * mNodalCoords[:, 1])[1])
        y = expand((H * mNodalCoords[:, 2])[1])

        dxdr = expand_derivatives(Dr(x))
        dxds = expand_derivatives(Ds(x))
        dydr = expand_derivatives(Dr(y))
        dyds = expand_derivatives(Ds(y))

        J = [dxdr dydr; dxds dyds] # Jacobiano
        J_inv = inv(J) # Inversa del Jacobiano
        J_det = det(J) # Determinante del Jacobiano
    
        B1 = transpose(∇H) * J_inv
        
        BL₀ = [B1[1, 1] 0 B1[2, 1] 0 B1[3, 1] 0 B1[4, 1] 0
            0 B1[1, 2] 0 B1[2, 2] 0 B1[3, 2] 0 B1[4, 2]
            B1[1, 2] B1[1, 1] B1[2, 2] B1[2, 1] B1[3, 2] B1[3, 1] B1[4, 2] B1[4, 1]]

        for i in 1:9
            Jg = substitute(J, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            Jg_inv = substitute(J_inv, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            Jg_det = substitute(J_det, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            Bg = substitute(BL₀, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
            KE += (transpose(Bg) * C * Bg * Jg_det * wGL9Q[i, 1] * wGL9Q[i, 2])
        end

        for i in 1:numGLE
            for j in 1:numGLE
                KG[GLE[i], GLE[j]] += KE[i, j].val
            end
        end
    end
    return KG
end

matrizKGL (generic function with 1 method)

In [13]:
KG = matrizKGL(mConectividadGlobal, mCoordNodales, C, numElementos, GDLGlobales, mGdLG, GL9Q, wGL9Q)

18×18 Matrix{Float64}:
      6.92308e5       2.88462e5  …       0.0             0.0
      2.88462e5       6.92308e5          0.0             0.0
     -4.61538e5  -57692.3                0.0             0.0
  57692.3             1.15385e5          0.0             0.0
      0.0             0.0                0.0             0.0
      0.0             0.0        …       0.0             0.0
      1.15385e5   57692.3                0.0             0.0
 -57692.3            -4.61538e5          0.0             0.0
     -3.46154e5      -2.88462e5         -3.46154e5      -2.88462e5
     -2.88462e5      -3.46154e5         -2.88462e5      -3.46154e5
      0.0             0.0        …       1.15385e5   57692.3
      0.0             0.0           -57692.3            -4.61538e5
      0.0             0.0                0.0             0.0
      0.0             0.0                0.0             0.0
      0.0             0.0               -4.61538e5  -57692.3
      0.0             0.0        …   57692.3

In [14]:
#RE = zeros(Number, GDLGlobales, numElementos)
RG = zeros(GDLGlobales)
RG[18] = 3000
RG[14] = 3000
#RG[8] = 3000
#RG[6] = 3000
#RG = [0., 3000., 0., 3000., 0., 0., 0., 0.]
display(RG)

18-element Vector{Float64}:
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
    0.0
 3000.0
    0.0
    0.0
    0.0
 3000.0

In [15]:
# Caso 1 (1 elemento)

# Imponemos condiciones de contorno de Dirichlet
#=
# U1x=0 => U1=0
KG[1, :] .= 0
KG[:, 1] .= 0
KG[1, 1] = 1

# U1y=0 => U2=0
KG[2, :] .= 0
KG[:, 2] .= 0
KG[2, 2] = 1

# U2x=0 => U3=0
KG[3, :] .= 0
KG[:, 3] .= 0
KG[3, 3] = 1

# U2y=0 => U4=0
KG[4, :] .= 0
KG[:, 4] .= 0
KG[4, 4] = 1

display(KG)

U₀ = KG \ RG # U₀ -> Desplazamientos en ⁰U 
=#

In [16]:
# Caso 2 (4 Elementos)
# Imponemos condiciones de contorno de Dirichlet

#
# U1x=0 => U1=0
KG[1, :] .= 0
KG[:, 1] .= 0
KG[1, 1] = 1

# U1y=0 => U2=0
KG[2, :] .= 0
KG[:, 2] .= 0
KG[2, 2] = 1

# U2x=0 => U3=0
KG[3, :] .= 0
KG[:, 3] .= 0
KG[3, 3] = 1

# U2y=0 => U4=0
KG[4, :] .= 0
KG[:, 4] .= 0
KG[4, 4] = 1

# U3x=0 => U5=0
KG[5, :] .= 0
KG[:, 5] .= 0
KG[5, 5] = 1

# U3y=0 => U6=0
KG[6, :] .= 0
KG[:, 6] .= 0
KG[6, 6] = 1

#display(KG)

U = KG \ RG
#

18-element Vector{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.00038012514028317975
  0.0024876583744940484
 -3.8883276355802194e-19
  0.0013895381141987445
 -0.00038012514028318035
  0.002487658374494048
  0.0023806785862530177
  0.007497264615256223
  8.325420909912773e-20
  0.0012774191848092266
 -0.0023806785862530177
  0.007497264615256221

In [19]:
# mGdLG <- matrix Grados de libertad global

mNodalCoords = zeros(4, 2)
mNodalCoordsₜ = zeros(4, 2)
FG = zeros(Float64, GDLGlobales)

for el in 1:numElementos
    GLE = mGdLG[:, el]
    numGLE = length(GLE)
    Uₑ = transpose(reshape(U[GLE], 2, 4))
    Fₑ = zeros(Float64, 8, 1)
    mConectividadElemental = mConectividadGlobal[el, :]
    for nodo in 1:4
        mNodalCoords[nodo, :] = mCoordNodales[mConectividadElemental[nodo], :]
    end

    x₀ = expand((H * mNodalCoords[:, 1])[1])
    y₀ = expand((H * mNodalCoords[:, 2])[1])

    dxdr = expand_derivatives(Dr(x₀))
    dxds = expand_derivatives(Ds(x₀))
    dydr = expand_derivatives(Dr(y₀))
    dyds = expand_derivatives(Ds(y₀))

    J = [dxdr dydr; dxds dyds] # Jacobiano
    J_inv = inv(J) # Inversa del Jacobiano
    J_det = det(J) # Determinante del Jacobiano


    B1 = J_inv * ∇H


    BL₀ = [B1[1, 1] 0 B1[1, 2] 0 B1[1, 3] 0 B1[1, 4] 0
        0 B1[2, 1] 0 B1[2, 2] 0 B1[2, 3] 0 B1[2, 4]
        B1[2, 1] B1[1, 1] B1[2, 2] B1[1, 2] B1[2, 3] B1[1, 3] B1[2, 4] B1[1, 4]]
    


        l₁₁, l₁₂, l₂₁, l₂₂= 0, 0, 0, 0

        for k in 1:4 l₁₁ += B1[1, k] * Uₑ[k, 1] end

        for k in 1:4 l₁₂ += B1[2, k] * Uₑ[k, 1] end

        for k in 1:4 l₂₁ += B1[1, k] * Uₑ[k, 2] end

        for k in 1:4 l₂₂ += B1[2, k] * Uₑ[k, 2] end


    BL₁₁ = [l₁₁ * B1[1, 1] l₂₁ * B1[1, 1] l₁₁ * B1[1, 2] l₂₁ * B1[1, 2]  l₁₁ * B1[1, 3] l₂₁ * B1[1, 3] l₁₁ * B1[1, 4] l₂₁ * B1[1, 4]]
    BL₁₂ = [l₁₂ * B1[2, 1] l₂₂ * B1[2, 1] l₁₂ * B1[2, 2] l₂₂ * B1[2, 2]  l₁₂ * B1[2, 3] l₂₂ * B1[2, 3] l₁₂ * B1[2, 4] l₂₂ * B1[2, 4]]
    BL₁₃ = [(l₁₁ * B1[2, 1] + l₁₂ * B1[1, 1]) (l₂₁ * B1[2, 1] + l₂₂ * B1[1, 1]) (l₁₁ * B1[2, 2] + l₁₂ * B1[1, 2]) (l₂₁ * B1[2, 2] + l₂₂ * B1[1, 2]) (l₁₁ *  B1[2, 3] + l₁₂ * B1[1, 3]) (l₂₁ * B1[2, 3] + l₂₂ * B1[1, 3]) (l₁₁ * B1[2, 4] + l₁₂ * B1[1, 4]) (l₂₁ * B1[2, 4] + l₂₂ * B1[1, 4])]
    Bₗ₁ = vcat(BL₁₁, BL₁₂, BL₁₃)
    #display(Bₗ₁)
    Bₗ = BL₀ + Bₗ₁

    # TENSOR GRADIENTE DE DEFORMACIONES

    X₁ = J_inv

    for nodo in 1:4
        mNodalCoordsₜ[nodo, :] = mCoordNodales[mConectividadElemental[nodo], :] + Uₑ[nodo, :]
    end
    xₜ = expand((H * mNodalCoordsₜ[:, 1])[1])
    yₜ = expand((H * mNodalCoordsₜ[:, 2])[1])

    dxdrₜ = expand_derivatives(Dr(xₜ))
    dxdsₜ = expand_derivatives(Ds(xₜ))
    dydrₜ = expand_derivatives(Dr(yₜ))
    dydsₜ = expand_derivatives(Ds(yₜ))

    X₂ = [dxdrₜ dydrₜ; dxdsₜ dydsₜ]

    X = X₂ * X₁ # X -> TENSOR GRADIENTE DE DEFORMACIONES

    ϵ = 1 / 2 * (transpose(X) * X - Matrix(I, 2, 2)) # TENSOR DE DEFORMACIONES DE GREEN-LAGRANGE

    ϵv = [ϵ[1,1]; ϵ[2,2]; ϵ[1,2]]

    S = C * ϵv # S -> SEGUNDO TENSOR DE PIOLA KIRCHHOFF
    for i in 1:9
        Sₑₗ = substitute(S, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
        Bₗₑₗ = substitute(Bₗ, Dict([r => GL9Q[i, 1], s => GL9Q[i, 2]]))
        Fₑ += transpose(Bₗₑₗ) * Sₑₗ
    end

    for i in 1:numGLE # numGLE -> Grados de libertad del elemento
            FG[GLE[i]] += Fₑ[i].val
    end
end
display(FG)
RG - FG

8×8 Matrix{Num}:
 -4.22361e-5(-4.22361e-5(8.07692e5(-1 + 1.77778((9.50313e-5 - 9.50313e-5r)^2) + 1.77778((0.749905 - 9.50313e-5s)^2)) + 3.46154e5(-1 + 1.77778((0.750969 - 0.00027453r)^2) + 1.77778((-0.00027453 - 0.00027453s)^2)))*((1 + s)^2) + 76923.1(1 + r)*(-1.29611e-19(1 + r) + 0.000126708(1 - r))*(1.77778(0.750969 - 0.00027453r)*(9.50313e-5 - 9.50313e-5r) + 1.77778(-0.00027453 - 0.00027453s)*(0.749905 - 9.50313e-5s)))*((1 + s)^2) + 0.333333(1 + r)*(-1.29611e-19(1 + r) + 0.000126708(1 - r))*(0.333333(1 + r)*(-1.29611e-19(1 + r) + 0.000126708(1 - r))*(3.46154e5(-1 + 1.77778((9.50313e-5 - 9.50313e-5r)^2) + 1.77778((0.749905 - 9.50313e-5s)^2)) + 8.07692e5(-1 + 1.77778((0.750969 - 0.00027453r)^2) + 1.77778((-0.00027453 - 0.00027453s)^2))) - 9.7468(1.77778(0.750969 - 0.00027453r)*(9.50313e-5 - 9.50313e-5r) + 1.77778(-0.00027453 - 0.00027453s)*(0.749905 - 9.50313e-5s))*((1 + s)^2))           …  -0.000122013(-4.22361e-5(8.07692e5(-1 + 1.77778((9.50313e-5 - 9.50313e-5r)^2) + 1.77778((0.7499

807692.3076923077(-1 + 1.7777777777777777((9.503128507076797e-5 - 9.503128507076797e-5r)^2) + 1.7777777777777777((0.7499049687149293 - 9.503128507076797e-5s)^2)) + 346153.8461538462(-1 + 1.7777777777777777((0.7509692991221733 - 0.00027453006507377475r)^2) + 1.7777777777777777((-0.00027453006507377475 - 0.00027453006507377475s)^2))

8×8 Matrix{Num}:
 -4.22361e-5(-4.22361e-5(8.07692e5(-1 + 1.77778((-9.50313e-5 - 9.50313e-5r)^2) + 1.77778((0.749905 - 9.50313e-5s)^2)) + 3.46154e5(-1 + 1.77778((0.750969 + 0.00027453r)^2) + 1.77778((0.00027453 + 0.00027453s)^2)))*((1 + s)^2) + 76923.1(1 + r)*(-0.000126708(1 + r) - 1.29611e-19(1 - r))*(1.77778(-9.50313e-5 - 9.50313e-5r)*(0.750969 + 0.00027453r) + 1.77778(0.749905 - 9.50313e-5s)*(0.00027453 + 0.00027453s)))*((1 + s)^2) + 0.333333(1 + r)*(-0.000126708(1 + r) - 1.29611e-19(1 - r))*(0.333333(1 + r)*(-0.000126708(1 + r) - 1.29611e-19(1 - r))*(3.46154e5(-1 + 1.77778((-9.50313e-5 - 9.50313e-5r)^2) + 1.77778((0.749905 - 9.50313e-5s)^2)) + 8.07692e5(-1 + 1.77778((0.750969 + 0.00027453r)^2) + 1.77778((0.00027453 + 0.00027453s)^2))) - 9.7468(1.77778(-9.50313e-5 - 9.50313e-5r)*(0.750969 + 0.00027453r) + 1.77778(0.749905 - 9.50313e-5s)*(0.00027453 + 0.00027453s))*((1 + s)^2))           …  0.000122013(-4.22361e-5(8.07692e5(-1 + 1.77778((-9.50313e-5 - 9.50313e-5r)^2) + 1.77778((0.7499

807692.3076923077(-1 + 1.7777777777777777((-9.503128507082348e-5 - 9.503128507082348e-5r)^2) + 1.7777777777777777((0.7499049687149292 - 9.503128507082348e-5s)^2)) + 346153.8461538462(-1 + 1.7777777777777777((0.7509692991221733 + 0.00027453006507377475r)^2) + 1.7777777777777777((0.00027453006507377475 + 0.00027453006507377475s)^2))

8×8 Matrix{Num}:
 0.333333(0.333333(3.46154e5(-1 + 1.77778((0.751224 - 0.00128043r)^2) + 1.77778((-0.00182949 - 0.00128043s)^2)) + 8.07692e5(-1 + 1.77778((0.000500138 - 0.000500138r)^2) + 1.77778((0.74931 - 0.000500138s)^2)))*(1 + s)*(-0.00079356(1 + s) - 0.000126708(1 - s)) + 76923.1(1 + r)*(1.57362e-19(1 + r) + 0.000666851(1 - r))*(1.77778(0.000500138 - 0.000500138r)*(0.751224 - 0.00128043r) + 1.77778(0.74931 - 0.000500138s)*(-0.00182949 - 0.00128043s)))*(1 + s)*(-0.00079356(1 + s) - 0.000126708(1 - s)) + 0.333333(1 + r)*(1.57362e-19(1 + r) + 0.000666851(1 - r))*(0.333333(1 + r)*(1.57362e-19(1 + r) + 0.000666851(1 - r))*(8.07692e5(-1 + 1.77778((0.751224 - 0.00128043r)^2) + 1.77778((-0.00182949 - 0.00128043s)^2)) + 3.46154e5(-1 + 1.77778((0.000500138 - 0.000500138r)^2) + 1.77778((0.74931 - 0.000500138s)^2))) + 76923.1(1.77778(0.000500138 - 0.000500138r)*(0.751224 - 0.00128043r) + 1.77778(0.74931 - 0.000500138s)*(-0.00182949 - 0.00128043s))*(1 + s)*(-0.00079356(1 + s) - 0.000126708(1 -

346153.8461538462(-1 + 1.7777777777777777((0.7512243718278431 - 0.0012804312925380201r)^2) + 1.7777777777777777((-0.0018294914226854586 - 0.0012804312925380201s)^2)) + 807692.3076923077(-1 + 1.7777777777777777((0.0005001383614924859 - 0.0005001383614924859r)^2) + 1.7777777777777777((0.7493097990683659 - 0.0005001383614924859s)^2))

8×8 Matrix{Num}:
 0.333333(0.333333(3.46154e5(-1 + 1.77778((0.751224 + 0.00128043r)^2) + 1.77778((0.00182949 + 0.00128043s)^2)) + 8.07692e5(-1 + 1.77778((-0.000500138 - 0.000500138r)^2) + 1.77778((0.74931 - 0.000500138s)^2)))*(1 + s)*(-0.00079356(1 + s) - 0.000126708(1 - s)) + 76923.1(1 + r)*(-0.000666851(1 + r) + 1.57362e-19(1 - r))*(1.77778(0.751224 + 0.00128043r)*(-0.000500138 - 0.000500138r) + 1.77778(0.74931 - 0.000500138s)*(0.00182949 + 0.00128043s)))*(1 + s)*(-0.00079356(1 + s) - 0.000126708(1 - s)) + 0.333333(1 + r)*(-0.000666851(1 + r) + 1.57362e-19(1 - r))*(0.333333(1 + r)*(-0.000666851(1 + r) + 1.57362e-19(1 - r))*(8.07692e5(-1 + 1.77778((0.751224 + 0.00128043r)^2) + 1.77778((0.00182949 + 0.00128043s)^2)) + 3.46154e5(-1 + 1.77778((-0.000500138 - 0.000500138r)^2) + 1.77778((0.74931 - 0.000500138s)^2))) + 76923.1(1.77778(0.751224 + 0.00128043r)*(-0.000500138 - 0.000500138r) + 1.77778(0.74931 - 0.000500138s)*(0.00182949 + 0.00128043s))*(1 + s)*(-0.00079356(1 + s) - 0.000126708(

346153.8461538462(-1 + 1.7777777777777777((0.7512243718278431 + 0.0012804312925380201r)^2) + 1.7777777777777777((0.0018294914226856251 + 0.0012804312925380201s)^2)) + 807692.3076923077(-1 + 1.7777777777777777((-0.0005001383614924304 - 0.0005001383614924304r)^2) + 1.7777777777777777((0.7493097990683659 - 0.0005001383614924304s)^2))

18-element Vector{Float64}:
  -2188.475476876391
  -6660.360106838581
      3.3151081879623234e-10
 -10730.46580357561
   2188.4754768761622
  -6660.360106838325
  -1162.1609818954344
  -1626.3907263621431
      9.179075277643278e-10
   3113.8516892009443
   1162.1609818940037
  -1626.3907263623023
   1312.0073928475294
  11061.668975738674
      1.8280843505635858e-10
   2066.7778292984585
  -1312.007392847303
  11061.668975738878

18-element Vector{Float64}:
  2188.475476876391
  6660.360106838581
    -3.3151081879623234e-10
 10730.46580357561
 -2188.4754768761622
  6660.360106838325
  1162.1609818954344
  1626.3907263621431
    -9.179075277643278e-10
 -3113.8516892009443
 -1162.1609818940037
  1626.3907263623023
 -1312.0073928475294
 -8061.668975738674
    -1.8280843505635858e-10
 -2066.7778292984585
  1312.007392847303
 -8061.668975738878